In [1]:
from __future__ import division
from __future__ import print_function  
import warnings
import pandas as pd
import gsp_support as gsp
import matplotlib.pyplot as plt
import numpy as np
pd.options.plotting.backend = "matplotlib"
import datetime
output_folder = 'FINAL/daily/'
input_folder = 'FINAL/Disaggregation/stage2_filter/'

In [2]:
pd.options.plotting.backend = "matplotlib"

In [3]:
folder = 'FINAL/Regression/'

In [4]:
total = pd.read_csv(input_folder+'3 min disaggregated result FINAL with removed meters.csv')

In [5]:
total['x_Timestamp'] = pd.to_datetime(total['x_Timestamp'])
total = total.set_index('x_Timestamp')

In [6]:
total = total.loc[~total['meter'].isin(['BR11','BR22','BR51'])]

In [7]:
total['hour'] = total.index.hour

In [8]:
len(total['meter'].unique())

24

In [9]:
start_date = '2019-05-01'
end_date = '2019-10-31'

mask = (total.index > start_date) & (total.index <= end_date)
total = total.loc[mask]

In [12]:
hrs

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [13]:
# for evening

# totaln = total.loc[(total['hour']>=6) & (total['hour']<=12)]

# totaln = total.loc[total['hour'].isin([18,19,20,21,22,23,1,2,3,4,5])]

hrs = [i for i in range(6,18)]

totaln = total.loc[total['hour'].isin(hrs)]

In [14]:
totaln

,gsp,power,current,compressor,final_compressor,ac_power,ac_on,voltage,meter,m-y,hour
x_Timestamp,,,,,,,,,,,
2019-05-09 06:00:00,0.0,320.0,1.53,0,0,0.0,0,253.82,MH24,2019-05,6
2019-05-09 06:03:00,0.0,340.0,1.55,0,0,0.0,0,255.78,MH24,2019-05,6
2019-05-09 06:06:00,0.0,320.0,1.55,0,0,0.0,0,256.16,MH24,2019-05,6
2019-05-09 06:09:00,0.0,320.0,1.58,0,0,0.0,0,255.90,MH24,2019-05,6
2019-05-09 06:12:00,0.0,280.0,1.34,0,0,0.0,0,256.26,MH24,2019-05,6
...,...,...,...,...,...,...,...,...,...,...,...
2019-10-30 17:45:00,0.0,0.0,0.00,0,0,0.0,0,0.00,MH21,2019-10,17
2019-10-30 17:48:00,0.0,0.0,0.00,0,0,0.0,0,0.00,MH21,2019-10,17
2019-10-30 17:51:00,0.0,0.0,0.00,0,0,0.0,0,0.00,MH21,2019-10,17


In [15]:
df = totaln.groupby('meter').resample('D').sum()

In [16]:
df = df.reset_index()

In [17]:
df = df[['meter','x_Timestamp','ac_on']]

In [18]:
df['ac_on'] = (df['ac_on']*3)/60

In [19]:
df['ac_on'].describe()

count    3478.000000
mean        1.542496
std         2.162574
min         0.000000
25%         0.000000
50%         0.550000
75%         2.500000
max        12.000000
Name: ac_on, dtype: float64

In [409]:
df.to_csv('FINAL/Regression/day.csv')

In [10]:
#daily logs

hrs = []
met = []
date_ = []


grouped = totaln.groupby('meter')

for name, group in grouped:
    print(name)
    sub = grouped.get_group(name)
    date_list = list(np.unique(sub.index.date))
    
    for date in date_list:
        date_.append(date)
        met.append(name)
        s = sub.loc[sub.index.date==date]
        
        # daily consumption in kWh
        
        
        
        ac_on = s.loc[s['ac_on']==1]
        hrs.append((ac_on.shape[0]*3)/60)
        
        

BR06
BR07


KeyboardInterrupt: 

In [ ]:
daily_logs = pd.DataFrame({'Meter':met, 'Date':date_, 'ac_usage_hrs':hrs})

In [ ]:
# temperature, humidity and other vars

In [20]:
we = pd.read_csv('FINAL/Regression/final TH.csv')

In [21]:
we['Row Labels'] = pd.to_datetime(we['Row Labels'],format ='%d-%m-%Y')

In [22]:
we

,Row Labels,BR_OT,MH_OT,BR_H,MH_H,Indoor_T,Night_indoor_T
0,2019-05-01,33.95,33.22,46.71,17.29,37.0,31.4
1,2019-05-02,32.73,30.66,54.63,24.92,37.1,31.8
2,2019-05-03,30.35,31.43,51.88,25.67,35.0,31.0
3,2019-05-04,32.13,31.39,47.96,27.46,34.3,30.6
4,2019-05-05,32.16,33.18,41.63,34.00,35.7,31.1
...,...,...,...,...,...,...,...
179,2019-10-27,23.87,22.30,76.38,64.58,27.8,23.3
180,2019-10-28,23.48,23.62,78.04,52.75,27.8,22.5
181,2019-10-29,24.22,23.36,66.71,58.17,27.9,22.8
182,2019-10-30,22.44,22.35,62.63,60.17,27.6,23.1


In [23]:
web = we[['Row Labels','BR_OT','BR_H']]
wem = we[['Row Labels','MH_OT','MH_H']]


In [24]:
# read hourly weather data

wem = pd.read_excel(folder+'weather.xlsx', sheet_name='Mathura')
web = pd.read_excel(folder+'weather.xlsx', sheet_name='Bareily')


In [25]:
start_time = '06:00:00'
end_time = '18:00:00'

In [26]:
# rough

wem['Time'] = pd.to_datetime(wem['Time'],format='%H:%M:%S')
wem.set_index('Time',inplace=True)
wem = wem.between_time(start_time, end_time)
wem = wem.groupby('Date').mean()
wem.reset_index(inplace=True)
wem['Date'] = wem['Date'].apply(lambda x: x.replace(year = x.year + 17))

In [27]:
wem

,Date,Dry Bulb Temp,Rel Humidity
0,2019-05-01,36.276923,16.153846
1,2019-05-02,34.300000,21.538462
2,2019-05-03,34.830769,23.461538
3,2019-05-04,34.915385,24.153846
4,2019-05-05,35.730769,32.000000
...,...,...,...
179,2019-10-27,25.500000,55.769231
180,2019-10-28,27.053846,45.846154
181,2019-10-29,26.600000,48.384615
182,2019-10-30,26.007692,50.538462


In [28]:
# preprocessing hourly weather data

def weather_preprocess(df,start_time,end_time):
    df['Time'] = pd.to_datetime(df['Time'],format='%H:%M:%S')
    df.set_index('Time',inplace=True)
    df_ = df.between_time(start_time, end_time)
    df_ = df_.groupby('Date').mean()
    df_.reset_index(inplace=True)
    df_['Date'] = df_['Date'].apply(lambda x: x.replace(year = x.year + 17))
    return df_
    

In [29]:
# prepare weather data for nighttime(6 pm to 6 am) - we_night


wem = pd.read_excel(folder+'weather.xlsx', sheet_name='Mathura')
web = pd.read_excel(folder+'weather.xlsx', sheet_name='Bareily')

start_time = '18:00:00'
end_time = '06:00:00'
we_night_m = weather_preprocess(wem,start_time,end_time)
we_night_b = weather_preprocess(web,start_time,end_time)


# prepare weather data for daytime (6 am to 6 pm) - we_day
wem = pd.read_excel(folder+'weather.xlsx', sheet_name='Mathura')
web = pd.read_excel(folder+'weather.xlsx', sheet_name='Bareily')

start_time = '06:00:00'
end_time = '18:00:00'
we_day_m = weather_preprocess(wem,start_time,end_time)
we_day_b = weather_preprocess(web,start_time,end_time)



# merge the data with day and night AC data respectively (ac_night, ac_day)




In [32]:
we_day_b

,Date,Dry Bulb Temp,Rel Humidity
0,2019-05-01,35.869231,42.769231
1,2019-05-02,34.215385,55.384615
2,2019-05-03,31.300000,49.230769
3,2019-05-04,33.353846,45.000000
4,2019-05-05,33.307692,40.230769
...,...,...,...
179,2019-10-27,25.200000,71.384615
180,2019-10-28,24.569231,72.846154
181,2019-10-29,24.046154,65.461538
182,2019-10-30,24.230769,56.076923


In [33]:
# read the dataframe

ac_night = pd.read_csv('FINAL/Regression/final datasets/fnight.csv')
ac_day = pd.read_csv('FINAL/Regression/final datasets/fday.csv')

In [34]:
ac_day

,Unnamed: 0,meter,Date,ac_on,family_size,avg_age_family,expenditure,room_area,ac_floor,win_area,own_cooler
0,0,MH02,01-05-2019,0.00,6,31.17,20000,100,0,0,0
1,1,MH02,02-05-2019,0.60,6,31.17,20000,100,0,0,0
2,2,MH02,03-05-2019,0.35,6,31.17,20000,100,0,0,0
3,3,MH02,04-05-2019,11.60,6,31.17,20000,100,0,0,0
4,4,MH02,05-05-2019,11.70,6,31.17,20000,100,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3473,3473,BR45,26-10-2019,2.25,4,23.00,20000,180,1,48,1
3474,3474,BR45,27-10-2019,0.40,4,23.00,20000,180,1,48,1
3475,3475,BR45,28-10-2019,0.00,4,23.00,20000,180,1,48,1
3476,3476,BR45,29-10-2019,0.00,4,23.00,20000,180,1,48,1


In [35]:
ac_night['Date'] = pd.to_datetime(ac_night['Date'],format ='%d-%m-%Y')
ac_day['Date'] = pd.to_datetime(ac_day['Date'],format ='%d-%m-%Y')

In [36]:
# merge

ac_night_b = ac_night.loc[ac_night['meter'].str.contains('B')]
ac_night_m = ac_night.loc[ac_night['meter'].str.contains('M')]
ac_day_b = ac_day.loc[ac_day['meter'].str.contains('B')]
ac_day_m = ac_day.loc[ac_day['meter'].str.contains('M')]

In [37]:
ac_day_m

,Unnamed: 0,meter,Date,ac_on,family_size,avg_age_family,expenditure,room_area,ac_floor,win_area,own_cooler
0,0,MH02,2019-05-01,0.00,6,31.17,20000,100,0,0,0
1,1,MH02,2019-05-02,0.60,6,31.17,20000,100,0,0,0
2,2,MH02,2019-05-03,0.35,6,31.17,20000,100,0,0,0
3,3,MH02,2019-05-04,11.60,6,31.17,20000,100,0,0,0
4,4,MH02,2019-05-05,11.70,6,31.17,20000,100,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2448,2448,MH45,2019-10-26,0.00,8,40.63,15000,135,1,20,0
2449,2449,MH45,2019-10-27,0.85,8,40.63,15000,135,1,20,0
2450,2450,MH45,2019-10-28,0.50,8,40.63,15000,135,1,20,0
2451,2451,MH45,2019-10-29,0.00,8,40.63,15000,135,1,20,0


In [439]:
ac_night.describe()

,Unnamed: 0,ac_on
count,3478.000000,3478.000000
mean,1738.500000,3.593976
std,1004.156445,3.318188
min,0.000000,0.000000
25%,869.250000,0.000000
50%,1738.500000,3.250000
75%,2607.750000,6.500000
max,3477.000000,11.000000


ac_day

In [41]:
br_night = pd.merge(ac_night_b, we_night_b, left_on='Date', right_on='Date')
br_day = pd.merge(ac_day_b, we_day_b, left_on='Date', right_on='Date')

In [42]:
br_night

,Unnamed: 0,meter,Date,ac_on,family_size,avg_age_family,expenditure,room_area,ac_floor,win_area,own_cooler,Dry Bulb Temp,Rel Humidity
0,2453,BR06,2019-07-11,0.00,2,75.00,10000,165,0,0,0,32.215385,70.384615
1,2565,BR07,2019-07-11,3.40,3,53.00,60000,240,0,0,0,32.215385,70.384615
2,2677,BR08,2019-07-11,3.35,4,38.50,15000,80,0,4,0,32.215385,70.384615
3,2454,BR06,2019-07-12,0.00,2,75.00,10000,165,0,0,0,30.576923,83.615385
4,2566,BR07,2019-07-12,6.00,3,53.00,60000,240,0,0,0,30.576923,83.615385
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,3109,BR24,2019-10-30,0.00,8,26.88,15000,144,1,32,0,20.523077,70.000000
1021,3215,BR23,2019-10-30,0.00,5,20.20,15000,144,0,20,0,20.523077,70.000000
1022,3321,BR31,2019-10-30,0.00,4,45.00,10000,100,0,12,0,20.523077,70.000000
1023,3405,BR38,2019-10-30,0.00,6,36.83,20000,150,1,16,1,20.523077,70.000000


In [44]:
mh_night = pd.merge(ac_night_m, we_night_m, left_on='Date', right_on='Date')
mh_day = pd.merge(ac_day_m, we_night_m, left_on='Date', right_on='Date')

In [46]:
day = mh_day.append(br_day)

In [49]:
night = mh_night.append(br_night)

In [47]:
day

,Unnamed: 0,meter,Date,ac_on,family_size,avg_age_family,expenditure,room_area,ac_floor,win_area,own_cooler,Dry Bulb Temp,Rel Humidity
0,0,MH02,2019-05-01,0.00,6,31.17,20000,100,0,0,0,30.169231,18.384615
1,183,MH07,2019-05-01,0.00,4,30.50,12000,165,0,0,0,30.169231,18.384615
2,366,MH10,2019-05-01,0.55,5,23.20,30000,240,1,20,1,30.169231,18.384615
3,549,MH14,2019-05-01,0.20,5,32.60,20000,168,0,24,1,30.169231,18.384615
4,732,MH18,2019-05-01,0.50,4,16.50,10000,168,1,0,1,30.169231,18.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,3109,BR24,2019-10-30,1.25,8,26.88,15000,144,1,32,0,24.230769,56.076923
1021,3215,BR23,2019-10-30,0.00,5,20.20,15000,144,0,20,0,24.230769,56.076923
1022,3321,BR31,2019-10-30,1.30,4,45.00,10000,100,0,12,0,24.230769,56.076923
1023,3405,BR38,2019-10-30,0.45,6,36.83,20000,150,1,16,1,24.230769,56.076923


In [50]:
night

,Unnamed: 0,meter,Date,ac_on,family_size,avg_age_family,expenditure,room_area,ac_floor,win_area,own_cooler,Dry Bulb Temp,Rel Humidity
0,0,MH02,2019-05-01,4.05,6,31.17,20000,100,0,0,0,30.169231,18.384615
1,183,MH07,2019-05-01,2.50,4,30.50,12000,165,0,0,0,30.169231,18.384615
2,366,MH10,2019-05-01,1.35,5,23.20,30000,240,1,20,1,30.169231,18.384615
3,549,MH14,2019-05-01,1.75,5,32.60,20000,168,0,24,1,30.169231,18.384615
4,732,MH18,2019-05-01,1.70,4,16.50,10000,168,1,0,1,30.169231,18.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,3109,BR24,2019-10-30,0.00,8,26.88,15000,144,1,32,0,20.523077,70.000000
1021,3215,BR23,2019-10-30,0.00,5,20.20,15000,144,0,20,0,20.523077,70.000000
1022,3321,BR31,2019-10-30,0.00,4,45.00,10000,100,0,12,0,20.523077,70.000000
1023,3405,BR38,2019-10-30,0.00,6,36.83,20000,150,1,16,1,20.523077,70.000000


In [217]:
mh_night = mh_night.rename(columns={'x_Timestamp':'Date','MH_OT':'Temperature','MH_H':'Humidity'})
br = br.rename(columns={'x_Timestamp':'Date','BR_OT':'Temperature','BR_H':'Humidity'})

In [452]:
finaln = mh_night.append(br_night)
finald = mh_day.append(br_day)

In [458]:
finald

,Unnamed: 0,meter,x_Timestamp,ac_on,Date,Dry Bulb Temp,Rel Humidity
0,1025,MH02,2019-05-01,0.00,2019-05-01,30.169231,18.384615
1,1208,MH07,2019-05-01,0.00,2019-05-01,30.169231,18.384615
2,1391,MH10,2019-05-01,0.55,2019-05-01,30.169231,18.384615
3,1574,MH14,2019-05-01,0.20,2019-05-01,30.169231,18.384615
4,1757,MH18,2019-05-01,0.50,2019-05-01,30.169231,18.384615
...,...,...,...,...,...,...,...
1020,655,BR23,2019-10-30,0.00,2019-10-30,24.230769,56.076923
1021,762,BR24,2019-10-30,1.25,2019-10-30,24.230769,56.076923
1022,868,BR31,2019-10-30,1.30,2019-10-30,24.230769,56.076923
1023,952,BR38,2019-10-30,0.45,2019-10-30,24.230769,56.076923


In [459]:
# tocheck

check = pd.read_csv('FINAL/Regression/Regression Input - Daily.csv')

In [460]:
check['Date'] = pd.to_datetime(check['Date'],format ='%d-%m-%Y')

In [462]:
set(finald['Date']) - set(check['Date'])

{Timestamp('2019-08-19 00:00:00'),
 Timestamp('2019-10-03 00:00:00'),
 Timestamp('2019-10-04 00:00:00')}

In [463]:
finaln = finaln[['meter','Date','ac_on','Dry Bulb Temp','Rel Humidity']]
finald = finald[['meter','Date','ac_on','Dry Bulb Temp','Rel Humidity']]

In [467]:
finaln.to_csv('Night Data.csv')
finald.to_csv('Day Data.csv')

In [230]:
# day = final
# eve = final
night = final

In [466]:
finaln

,meter,Date,ac_on,Dry Bulb Temp,Rel Humidity
0,MH02,2019-05-01,4.05,30.169231,18.384615
1,MH07,2019-05-01,2.50,30.169231,18.384615
2,MH10,2019-05-01,1.35,30.169231,18.384615
3,MH14,2019-05-01,1.75,30.169231,18.384615
4,MH18,2019-05-01,1.70,30.169231,18.384615
...,...,...,...,...,...
1020,BR23,2019-10-30,0.00,20.523077,70.000000
1021,BR24,2019-10-30,0.00,20.523077,70.000000
1022,BR31,2019-10-30,0.00,20.523077,70.000000
1023,BR38,2019-10-30,0.00,20.523077,70.000000


In [231]:
eve

,meter,Date,ac_on,Temperature,Humidity
0,MH02,2019-05-01,0.00,33.22,17.29
1,MH07,2019-05-01,0.00,33.22,17.29
2,MH10,2019-05-01,0.55,33.22,17.29
3,MH14,2019-05-01,0.20,33.22,17.29
4,MH18,2019-05-01,0.00,33.22,17.29
...,...,...,...,...,...
1020,BR23,2019-10-30,0.00,22.44,62.63
1021,BR24,2019-10-30,1.25,22.44,62.63
1022,BR31,2019-10-30,0.05,22.44,62.63
1023,BR38,2019-10-30,0.00,22.44,62.63


In [468]:
# set the other features

socio = pd.read_excel('FINAL/Regression/socio.xlsx')

In [480]:
daily = pd.read_csv('FINAL/Regression/daily.csv')

In [500]:
daily_['hh']

0       MH02
1       MH02
2       MH02
3       MH02
4       MH02
        ... 
3496    BR45
3497    BR45
3498    BR45
3499    BR45
3500    BR45
Name: hh, Length: 3501, dtype: object

In [496]:
# merge with socio

# day_ = pd.merge(finald, socio, left_on='meter', right_on='hh')
# night_ = pd.merge(finaln, socio, left_on='meter', right_on='hh')
daily_ = pd.merge(daily, socio, left_on='meter', right_on='hh')

In [476]:
night_['ac_on'].describe()

count    3478.000000
mean        3.593976
std         3.318188
min         0.000000
25%         0.000000
50%         3.250000
75%         6.500000
max        11.000000
Name: ac_on, dtype: float64

In [477]:
day_ = day_.drop(['hh'],axis = 1)
night_ = night_.drop(['hh'],axis = 1)

KeyError: "['hh'] not found in axis"

In [501]:
daily_ = daily_.drop(['hh'],axis = 1)

In [486]:
day_.to_csv('FINAL/Regression/fday.csv')
night_.to_csv('FINAL/Regression/fnight.csv')

In [502]:
daily_.to_csv('FINAL/Regression/fdaily.csv')

In [492]:
socio = socio[['hh','room_area','ac_floor','win_area','own_cooler']]

In [493]:
socio

,hh,room_area,ac_floor,win_area,own_cooler
0,MH02,100,0,0,0
1,MH07,165,0,0,0
2,MH10,240,1,20,1
3,MH14,168,0,24,1
4,MH18,168,1,0,1
5,MH21,150,0,12,1
6,MH22,120,0,24,1
7,MH23,144,0,11,0
8,MH24,135,1,20,1
9,MH27,165,1,16,1
